In [4]:
!pip install pandas-profiling
pip install pydantic-settings



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas
from pandas_profiling import ProfileReport
import folium
from datetime import datetime

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.4/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.4/u/import-error

In [ ]:
# Load spareroom_listings which already has headers
data = pandas.read_csv('spareroom_listing.csv')


ValueError: cannot reshape array of size 61 into shape (5,5)

In [ ]:
# Filter spareroom_listings for listings with a price of 900 or less and bills included
profile = ProfileReport(data)
profile.to_notebook_iframe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 61 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   url                 30 non-null     object 
 1   id                  30 non-null     int64  
 2   title               30 non-null     object 
 3   desc                30 non-null     object 
 4   Type                30 non-null     object 
 5   Area                30 non-null     object 
 6   Postcode            30 non-null     object 
 7   Nearest station     19 non-null     object 
 8   location_coords     30 non-null     object 
 9   Available           30 non-null     object 
 10  Minimum term        26 non-null     object 
 11  Maximum term        4 non-null      object 
 12  room_0_price        30 non-null     int64  
 13  room_0_type         30 non-null     object 
 14  Deposit(room 1)     8 non-null      object 
 15  room_1_price        13 non-null     float64
 16  room_1_typ

In [ ]:
def create_popup_content(row):
    room_prices = []
    for i in range(3):
        price_key = f"room_{i}_price"
        type_key = f"room_{i}_type"
        if row[price_key] != "":
            room_prices.append(
                f"<li>Room {i + 1}: £{row[price_key]} ({row[type_key]})</li>"
            )
    return f"""
        <div style="width: 200px;">
            <h3>{row["Area"]}</h3>
            <p><strong>Type:</strong> {row["Type"]}</p>
            <p><strong>Available:</strong> {row["Available"]}</p>
            <p><strong>Room Prices:</strong></p>
            <ul>
                {''.join(room_prices)}
            </ul>
            <a href="{row["url"]}" target="_blank">View Details</a>
        </div>
    """

data.dtypes
# data["location_coords"] = data["location_coords"].apply(parse_coordinates)
data["Latitude"] = data["location_coords"].apply(lambda x: x[0])
data["Longitude"] = data["location_coords"].apply(lambda x: x[1])
# data["Available"] = data["Available"].apply(calculate_available_months)

# Find latitude mean from pandas column and assign to variable

longitudes_mean = data["Longitude"]

map_center = [data["Latitude"].mean(), data["Longitude"].mean()]
my_map = folium.Map(location=map_center, zoom_start=12, tiles="cartodbdark_matter")

for _, row in data.iterrows():
    popup_content = create_popup_content(row)
    icon_color = (
        "green"
        if row["date_scraped"] == datetime.today().strftime("%d-%m-%Y")
        else "orange"
    )
    icon = folium.Icon(
        color=icon_color, icon="star"
    )
    folium.Marker(
        location=(row["Latitude"], row["Longitude"]),
        popup=folium.Popup(popup_content, max_width=250),
        tooltip=row["Area"],
        icon=icon,
    ).add_to(my_map)
# Display the map
my_map

/var/folders/ns/9q89dwnd3q360t_ksqwyd8s80000gn/T/ipykernel_66332/983413363.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Latitude"] = data["location_coords"].apply(lambda x: x[0])
/var/folders/ns/9q89dwnd3q360t_ksqwyd8s80000gn/T/ipykernel_66332/983413363.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Longitude"] = data["location_coords"].apply(lambda x: x[1])
